In [7]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow_datasets as tfds

import tensorflow_datasets.public_api as tfds

import tf_keras as keras

from keras.layers import Dense, Dropout
from keras.models import Sequential
from keras import utils
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator

dataset = tfds.load('oxford_iiit_pet')

In [30]:
# Разделение на обучающий и тестовый набор
train_dataset = dataset['train']
test_dataset = dataset['test']

# Преобразование в numpy массивы
train_data = tfds.as_numpy(train_dataset)
test_data = tfds.as_numpy(test_dataset)

# Извлечение признаков и меток для обучающего набора

train_gen = ImageDataGenerator(rescale = 1./255,
                               rotation_range = 30,
                               width_shift_range = 0.1,                             
                               fill_mode = 'nearest',
                               )

img_size = (224, 224)
x_train = []
y_train = []
for example in train_data:
    image_resized = tf.image.resize_with_pad(example['image'], *img_size).numpy().astype("uint8")
    x_train.append(image_resized)
    y_train.append(example['label'])

x_train = np.array(x_train)
y_train = np.array(y_train)

tr_data = train_gen.flow(x = x_train, y = y_train, batch_size = 32, shuffle = False)

# Извлечение признаков и меток для тестового набора

val_gen = ImageDataGenerator(rescale = 1./255)

x_test = []
y_test = []
for example in test_data:
    image_resized = tf.image.resize_with_pad(example['image'], *img_size).numpy().astype("uint8")
    x_test.append(image_resized)  
    y_test.append(example['label'])

x_test = np.array(x_test)
y_test = np.array(y_test)

val_data = val_gen.flow(x=x_test[:730], y=y_test[:730],  batch_size = 32, shuffle = False)

Corrupt JPEG data: 240 extraneous bytes before marker 0xd9
Corrupt JPEG data: premature end of data segment


In [9]:
label = [
    "Abyssinian",
    "american_bulldog",
    "american_pit_bull_terrier",
    "basset_hound",
    "beagle",
    "Bengal",
    "Birman",
    "Bombay",
    "boxer",
    "British_Shorthair",
    "chihuahua",
    "Egyptian_Mau",
    "english_cocker_spaniel",
    "english_setter",
    "german_shorthaired",
    "great_pyrenees",
    "havanese",
    "japanese_chin",
    "keeshond",
    "leonberger",
    "Maine_Coon",
    "miniature_pinscher",
    "newfoundland",
    "Persian",
    "pomeranian",
    "pug",
    "Ragdoll",
    "Russian_Blue",
    "saint_bernard",
    "samoyed",
    "scottish_terrier",
    "shiba_inu",
    "Siamese",
    "Sphynx",
    "staffordshire_bull_terrier",
    "wheaten_terrier",
    "yorkshire_terrier",
]

In [ ]:
plt.subplots(nrows = 4, ncols = 4, figsize = (20, 20))

for i, imgIndex in enumerate(np.random.randint(0, len(x_train), size = 16)):
    plt.subplot(4, 4, i + 1) 
    plt.xticks([])
    plt.yticks([])
    plt.imshow(x_train[imgIndex])
    breed = label[y_train[imgIndex]]
    species = "cat" if breed[0].isupper() == True else "dog"
    plt.xlabel(f'{breed}\n{species}')
plt.show()

In [ ]:
import tensorflow_hub as hub
efficientnetv2_s = 'https://tfhub.dev/google/imagenet/efficientnet_v2_imagenet1k_s/classification/2'

pretrained_base = hub.KerasLayer(efficientnetv2_s, trainable = False)
model = keras.Sequential([
    pretrained_base,
    keras.layers.Flatten(),
    keras.layers.Dense(128,activation='relu'),
    keras.layers.Dropout(0.3),
    keras.layers.Dense(64,activation='relu'),
    keras.layers.Dropout(0.3),
    keras.layers.Dense(37, activation='softmax'), 
])
model.compile(
    optimizer='adam',
    loss = 'sparse_categorical_crossentropy',
    metrics=['accuracy'],
    steps_per_execution=1
)
model.fit(
    tr_data,
    epochs=3,
    validation_data=val_data,
    steps_per_epoch=180
)

Epoch 1/3
180/180 [==============================] - ETA: 0s - loss: 1.4084 - accuracy: 0.6262

In [34]:
predictions = [np.argmax(predict) for predict in model.predict(tr_data, steps=50)]
for i in range(len(predictions)): 
    print(predictions[i] == y_train[i])

50/50 [==============================] - 15s 313ms/step
False
False
False
False
False
True
False
False
False
False
False
False
False
True
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
True
False
False
False
False
True
False
False
False
False
False
False
False
False
True
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
F

In [ ]:
plt.subplots(nrows = 4, ncols = 4, figsize = (20, 20))

for i, imgIndex in enumerate(np.random.randint(0, len(x_train), size = 16)):
    plt.subplot(4, 4, i + 1) 
    plt.xticks([])
    plt.yticks([])
    plt.imshow(x_train[imgIndex])
    breed = label[y_train[imgIndex]]
    species = "cat" if breed[0].isupper() == True else "dog"
    plt.xlabel(f'{breed}\n{species}')
plt.show()